In [1]:
import torch
import numpy as np
import sklearn.neighbors as nn

In [2]:
input_ab = torch.load('input_ab.pt')
output_ab = torch.load('output_ab.pt')
inputs = [x for _,x in enumerate(input_ab)]
outputs = [x for _,x in enumerate(output_ab)]
# image distribution of probabilities
z_pred = outputs[0].detach().cpu().numpy()
z_pred.shape
# image true ab channel
z_true = inputs[0].detach().cpu().numpy()*255-128
z_true.shape

$w = \bigg((1-\lambda)\tilde{p} + \frac{\lambda}{Q}\bigg)^{-1}$
$E[w] = \sum_q \tilde{p}_q w_q = 1$

In [164]:
lambda_ = 0.5
p_tilde = np.load('prior_prob.npy')
q_stars = smoothed_normalized.argmax(axis=1)
w = 1/(lambda_ * p_tilde + lambda_/q)

# normalizing s.t. E[w] = 1
total = p_tilde @ w
p_tilde = p_tilde / total

# rebalancing weighting term
weight = w[q_stars]

$L_{cl}(\hat{Z},Z) = - \sum_{h,w} v(Z_{h,w}) \sum_q Z_{h,w,q} \log(\hat{Z}_{h,w,q})$

In [269]:
eps = 1e-5
z_pred_shifted = z_pred - z_pred.min(axis=0)
z_pred_shifted = z_pred_shifted + eps
z_pred_shifted = z_pred_shifted / z_pred_shifted.sum(axis=0)
z_pred_shifted = z_pred_shifted.reshape(-1, q)
v = (smoothed_normalized * np.log(z_pred_shifted)).sum(axis=1)
v = - v * weight
v.sum()

In [16]:
pts_hull = np.load('data/pts_in_hull.npy')
image_ab = z_true
q = 313
sigma = 5

In [38]:
nearest = nn.NearestNeighbors(n_neighbors=5, algorithm='ball_tree')
nearest.fit(pts_hull)

# flattened array of shape (224*224, 2)
image_a = z_true[0, :, :].ravel()
image_b = z_true[1, :, :].ravel()
image_ab = np.vstack((image_a, image_b)).T

# distances and indices of the 5 nearest neighbour of the true image ab channel
dists, ind = nearest.kneighbors(image_ab)

In [51]:
wts = np.exp(- dists ** 2) / (2 * sigma**2)
wts = wts / np.sum(wts, axis=1)[:, np.newaxis]
z_soft_encoding = np.zeros((image_ab.shape[0], q))

In [52]:
idx_pts = np.arange(image_ab.shape[0])[:, np.newaxis]
z_soft_encoding[idx_pts, ind] = wts

In [77]:
p_tilde = np.load('data/prior_prob_smoothed.npy')

from utils import *

mc = MultimodalClassification()
smoothed_normalized = mc.compute_smoothed(z_true)
q_star = smoothed_normalized.argmax(axis=1)